In [46]:
import deribit_data as dm
from option import Option
import numpy as np
from utils import *
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go   

In [47]:
data = dm.Scraper(currency='BTC')

In [48]:
raw_df = data.collect_data(max_workers = 15, save_csv = False)

Data Collected


In [49]:
#raw_df.head()

In [50]:
option = Option(raw_df, 'C')

E:\GitHub\deribit_scraping\utils.py:104: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

E:\GitHub\deribit_scraping\utils.py:128: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

E:\GitHub\deribit_scraping\utils.py:129: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

E:\GitH

In [51]:
#option.init_merton(m = 0.625038301534241, lam =0.0010591027301815744, v = 0.511097066109511,  )
option.init_merton(reset=True)
option.init_heston(reset=True)

In [52]:
option.append_imp_vol_to_df()
option.append_loc_vol_to_df()

E:\GitHub\deribit_scraping\option.py:212: RuntimeWarning:

divide by zero encountered in double_scalars

E:\GitHub\deribit_scraping\option.py:212: RuntimeWarning:

invalid value encountered in sqrt



In [53]:
option.append_BS_price()

In [54]:
option.df.dropna(inplace=True)
option.df = option.df.drop(option.df[option.df.BS_PRICE >= 5].index) #a revoir
#option.df = option.df.drop(option.df[option.df.IV_moneyness >= .5*10**6].index)

In [55]:
#option.optimize_merton(tol = 1e-6, max_iter = 600)



In [56]:
option.append_Merton_price()
option.append_Heston_prices()

In [57]:
x = option.df['S']/option.df['K']

#x = option.df['K']
y = option.df['_T']
z = option.df['L_VOL']
z2 = option.df['mark_iv']/100


fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z , mode='markers', name='Local vol')])
#fig = go.Figure(data=[go.Surface(x=x, y=y, z=z , name='Local vol')])
                      #go.Scatter3d(x=x, y=y, z=z2 , mode='markers', name='Implied vol deribit')])
"""
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z , mode='markers', name='Black&Scholes model Price'),
                      go.Scatter3d(x=x, y=y, z=z2, mode='markers', name='Real Market Price')])
"""

fig.update_scenes(xaxis_title_text='S/K', yaxis_title_text='Exp', zaxis_title_text='IV') 
fig.show()

In [58]:
#LINEAR INTERPOLATION
new_df = option.df[['moneyness', '_T', 'I_VOL_BS']]
new_df = new_df.pivot(index = '_T',columns='moneyness', values='I_VOL_BS')

new_df=new_df.interpolate(method='linear',axis=1,limit_direction='both')
z = new_df
x = list(new_df.columns)
y = list(new_df.index)


fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
fig.update_layout(title='Implied Volatility', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.update_scenes(xaxis_title_text='S/K', yaxis_title_text='Exp', zaxis_title_text='IV') 
fig.show()

In [61]:
#LINEAR INTERPOLATION
new_df = option.df[['moneyness', '_T', 'L_VOL']]
new_df = new_df.pivot(index = '_T',columns='moneyness', values='L_VOL')

new_df=new_df.interpolate(method='linear',axis=1,limit_direction='both')
z = new_df
x = list(new_df.columns)
y = list(new_df.index)


fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
fig.update_layout(title='Local Volatility', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))

fig.update_scenes(xaxis_title_text='S/K', yaxis_title_text='Exp', zaxis_title_text='Local Vol') 
fig.show()

In [62]:

new_df = option.df[['K', '_T', 'L_VOL']]
new_df = new_df.pivot(index = '_T',columns='K', values='L_VOL')

new_df=new_df.interpolate(method='spline',order=2,axis=1)
z = new_df
x = list(new_df.columns)
y = list(new_df.index)


fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
fig.update_layout(title='xxx', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()


error: (m>k) failed for hidden m: fpcurf0:m=2

In [ ]:
#option.init_merton(m = 0.625038301534241, lam =0.0010591027301815744, v = 0.511097066109511,  )
option.init_merton(reset=True)
option.append_Merton_price()
rms = mean_squared_error(option.df['MERTON_PRICE'], option.df['mid'], squared=False)
print('Root Mean Squared Error = {}%'.format(round(rms*100,3)))

In [63]:
option.df.dropna(inplace=True)
option.df = option.df.drop(option.df[option.df._T >= .08].index)

In [67]:
x = option.df['K']
y = option.df['_T']
z = option.df['BS_PRICE']
z2 = option.df['mid']
z3 = option.df['MERTON_PRICE']
z4 = option.df['HESTON_PRICE']


#fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z , mode='markers', name='BS price')])

fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z , mode='markers', name='Black&Scholes model Price', marker=dict(opacity=0.8)),
                      go.Scatter3d(x=x, y=y, z=z2, mode='markers', name='Real Market Price', marker=dict(opacity=0.8)),
                      go.Scatter3d(x=x, y=y, z=z3, mode='markers', name='Merton Model Price', marker=dict(opacity=0.8),),
                      go.Scatter3d(x=x, y=y, z=z4, mode='markers', name='Heston Model Price', marker=dict(opacity=0.8))])


fig.update_scenes(xaxis_title_text='Strike', yaxis_title_text='Exp', zaxis_title_text='Price') 
fig.show()

rms_merton = mean_squared_error(option.df['mid'], option.df['MERTON_PRICE'], squared=False)
rms_bs     = mean_squared_error(option.df['mid'], option.df['BS_PRICE'], squared=False)
rms_heston = mean_squared_error(option.df['mid'], option.df['HESTON_PRICE'], squared=False)

print('Root Mean Squared Error | MERTON = {} %'.format(round(rms_merton*100,6)))
print('Root Mean Squared Error | B&S    = {} %'.format(round(rms_bs*100,6)))
print('Root Mean Squared Error | HESTON    = {} %'.format(round(rms_heston*100,6)))

Root Mean Squared Error | MERTON = 0.780813 %
Root Mean Squared Error | B&S    = 0.168169 %
Root Mean Squared Error | HESTON    = 0.543592 %


## Test optim Heston

In [65]:
x0 = np.array([0.08, 0.10, 0.8, 3, 0.25])
res = option.optimize_heston(x0,tol = 1e-5, max_iter =300 )

     fun: 0.026069755825197646
     jac: array([-0.00067326,  0.00097389,  0.00022047, -0.00024913,  0.00063245])
 message: 'Optimization terminated successfully.'
    nfev: 36
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([0.93535951, 0.14206404, 0.8073167 , 2.99901601, 0.27319069])


In [66]:
option.init_heston(theta = res)
option.append_Heston_prices()